## Validation of perceptron

In [21]:
%load_ext autoreload
%autoreload 2
import random
from perceptron import Perceptron
import numpy as np
from sklearn.datasets import load_digits

from sklearn.model_selection import train_test_split


X, y = load_digits(return_X_y=True, n_class=2)

# Split in 60/40
Xtrain, Xdata, yTrain, yData = train_test_split(X, y * 2 - 1, test_size=0.2, random_state=42, stratify=(y * 2 - 1))
# Split in 50/50 the remaining 40%
Xval, Xtest, yVal, yTest = train_test_split(Xdata, yData, test_size=0.5, random_state=42, stratify=yData)
random.seed(42)
accuracies = []
for i in range(100):
    initial_weights = np.random.randn(Xtrain.shape[1] + 1)
    for alpha in [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10, 10**1, 10**2, 10**3, 10**4]:
        perceptron = Perceptron(alpha)
        perceptron.train(Xtrain, yTrain, initial_weights)
        predictions = perceptron.predict(Xval)
        accuracy = np.mean(predictions == yVal)
        margin = perceptron.compute_min_margin(Xval)
        accuracies.append((initial_weights, alpha, accuracy, margin))
        
# Find the maximum accuracy
max_accuracy = max(accuracies, key=lambda x: x[2])[2]

# Filter accuracies based on the maximum accuracy
best = [el for el in accuracies if el[2] == max_accuracy]

print(best)
for el in best:
    print(f'Alpha: {el[1]} Accuracy: {el[2]} Margin: {el[3]}')

best_margin = max(best, key=lambda x: x[3])
print(f'Initial_weights: {best_margin[0]} Alpha: {best_margin[1]} Accuracy: {best_margin[2]} Margin: {best_margin[3]}')



                


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[(array([ 0.42858566,  0.51109683,  0.40766693, -0.48984857,  0.56420483,
       -0.90052489, -1.41744745,  0.17914316,  0.45332584,  0.40415245,
       -1.69542241,  0.55861796, -0.76012316,  0.70400065,  0.04065392,
        0.68029187,  0.85488654, -0.94634003, -0.82714977,  0.75992169,
       -2.04144221, -0.96460223, -0.98697079,  0.10546001,  0.8615789 ,
        0.16432577,  0.88510595,  0.33084704, -0.24486814,  0.04518038,
       -0.53245265, -0.45360454, -1.05251198, -0.7671583 , -0.64867762,
        1.24283664, -0.03427432, -0.67935626, -0.478827  ,  1.81634877,
        0.79357264, -0.40017933, -2.128659  , -1.0689421 ,  0.23682649,
       -0.45533483, -0.49388219, -0.69668812, -0.4434462 ,  0.86116376,
       -1.70930682,  1.64832829, -0.55705442, -2.0484381 , -0.77262708,
        1.17869268, -0.40521235, -0.07004933, -0.33132024,  0.02038523,
       -0.16770074, -0.38380555, -2.44560266, 